

---

# **HackMty_Yolov11n**
## **Seccion 1: Imports de database y de dependencias**


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.1 MB/s eta 0:00:00


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="1kyEcz6MV5H8ZK9UIIBP")
project = rf.workspace("jorgelop").project("pothole-detection-using-yolov5-qxgyv")
version = project.version(2)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 142.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Pothole-detection-using-YOLOv5-2 in yolov8:: 100%|██████████| 1342/1342 [00:00<00:00, 5048.89it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## **Seccion 2:**

Aqui solamente nos aseguramos que la estructura del dataset sea la adecuada, contando con los distintos archivos necearios para el entrenamiento y la prueba


In [ ]:
import os
from pathlib import Path

# Verificar estructura del dataset
dataset_path = Path('/content/pothole-18')

print("📁 Estructura del dataset:")
print("\nArchivos en la raíz:")
for item in dataset_path.iterdir():
    print(f"  - {item.name}")

# Verificar carpetas específicas
folders_to_check = ['train', 'valid', 'test', 'val']
for folder in folders_to_check:
    folder_path = dataset_path / folder
    if folder_path.exists():
        print(f"\n✅ Carpeta '{folder}' existe:")
        for subfolder in folder_path.iterdir():
            if subfolder.is_dir():
                num_files = len(list(subfolder.iterdir()))
                print(f"  - {subfolder.name}: {num_files} archivos")
    else:
        print(f"\n❌ Carpeta '{folder}' NO existe")

# Verificar data.yaml
yaml_path = dataset_path / 'data.yaml'
if yaml_path.exists():
    print("\n📄 Contenido de data.yaml:")
    print("-" * 50)
    with open(yaml_path, 'r') as f:
        print(f.read())
    print("-" * 50)
else:
    print("\n❌ data.yaml NO existe")

📁 Estructura del dataset:

Archivos en la raíz:
  - README.roboflow.txt
  - train
  - data.yaml
  - README.dataset.txt

✅ Carpeta 'train' existe:
  - labels: 1302 archivos
  - images: 1302 archivos

❌ Carpeta 'valid' NO existe

❌ Carpeta 'test' NO existe

❌ Carpeta 'val' NO existe

📄 Contenido de data.yaml:
--------------------------------------------------
names:
- pothole
nc: 1
roboflow:
  license: CC BY 4.0
  project: pothole-vhmow
  url: https://universe.roboflow.com/yeeun-kim-fyvoj/pothole-vhmow/dataset/18
  version: 18
  workspace: yeeun-kim-fyvoj
test: ../test/images
train: ../train/images
val: ../valid/images

--------------------------------------------------


## **Seccion 3**

En caso de que no, correr esto para tomar cierta cantidad de los valores de train y distribuirlos de una forma en la que se puedan utilizar para el validation dataset

In [ ]:
import os
import shutil
from pathlib import Path
import random

# Rutas
dataset_path = Path('/content/pothole-18')
train_images = dataset_path / 'train' / 'images'
train_labels = dataset_path / 'train' / 'labels'

# Crear carpetas de validación
valid_images = dataset_path / 'valid' / 'images'
valid_labels = dataset_path / 'valid' / 'labels'
valid_images.mkdir(parents=True, exist_ok=True)
valid_labels.mkdir(parents=True, exist_ok=True)

# Obtener todas las imágenes
all_images = list(train_images.glob('*.jpg')) + list(train_images.glob('*.png')) + list(train_images.glob('*.jpeg'))
print(f"📊 Total de imágenes encontradas: {len(all_images)}")

# Shuffle con seed para reproducibilidad
random.seed(42)
random.shuffle(all_images)

# Split 85/15 (te quedas con más datos para entrenar)
split_ratio = 0.85
split_idx = int(len(all_images) * split_ratio)
valid_set = all_images[split_idx:]

print(f"\n🔄 Moviendo imágenes a validación...")
moved_images = 0
moved_labels = 0

# Mover archivos a validación
for img_path in valid_set:
    # Mover imagen
    dest_img = valid_images / img_path.name
    shutil.move(str(img_path), str(dest_img))
    moved_images += 1

    # Mover label correspondiente
    label_name = img_path.stem + '.txt'
    label_path = train_labels / label_name
    if label_path.exists():
        dest_label = valid_labels / label_name
        shutil.move(str(label_path), str(dest_label))
        moved_labels += 1

print(f"\n✅ Split completado:")
print(f"   📁 Train: {len(list(train_images.glob('*.*')))} imágenes")
print(f"   📁 Valid: {len(list(valid_images.glob('*.*')))} imágenes")
print(f"   📄 Labels train: {len(list(train_labels.glob('*.txt')))}")
print(f"   📄 Labels valid: {len(list(valid_labels.glob('*.txt')))}")
print(f"\n📊 Proporción: {split_ratio*100:.0f}% train / {(1-split_ratio)*100:.0f}% valid")

📊 Total de imágenes encontradas: 1302

🔄 Moviendo imágenes a validación...

✅ Split completado:
   📁 Train: 1106 imágenes
   📁 Valid: 196 imágenes
   📄 Labels train: 1106
   📄 Labels valid: 196

📊 Proporción: 85% train / 15% valid


## **Seccion 5**

In [ ]:
import yaml

# Actualizar data.yaml
data_yaml = {
    'path': '/content/pothole-18',
    'train': 'train/images',
    'val': 'valid/images',

    'nc': 1,
    'names': ['pothole']
}

yaml_path = '/content/pothole-18/data.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False, sort_keys=False)

print("✅ data.yaml actualizado")
print("\n📄 Nuevo contenido:")
with open(yaml_path, 'r') as f:
    print(f.read())

✅ data.yaml actualizado

📄 Nuevo contenido:
path: /content/pothole-18
train: train/images
val: valid/images
nc: 1
names:
- pothole



## **Seccion 6**

Entrenamiento/fine-tunning de una arquitectura YOLO, corriendo durante un total de 30 epocas, los demas parametros estas descritos dentro del codigo

In [ ]:
from ultralytics import YOLO

# Cargar modelo preentrenado
model = YOLO('yolov8n.pt')

# Entrenar
results = model.train(
    data='/content/Pothole-detection-using-YOLOv5-2/data.yaml',
    epochs=30,
    imgsz=640,
    batch=16,
    patience=10,          # early stopping si no mejora en 10 epochs
    device='cpu',             # GPU
    workers=8,
    project='pothole_detection',
    name='yolov8n_v1',
    exist_ok=True,

    # Optimizaciones recomendadas
    optimizer='AdamW',    # mejor que SGD para datasets pequeños
    lr0=0.001,           # learning rate inicial
    lrf=0.01,            # learning rate final
    momentum=0.937,
    weight_decay=0.0005,

    # Augmentations (ya vienen por default pero puedes ajustar)
    hsv_h=0.015,         # hue
    hsv_s=0.7,           # saturation
    hsv_v=0.4,           # value
    degrees=10.0,        # rotación
    translate=0.1,       # traslación
    scale=0.5,           # zoom
    flipud=0.0,          # no voltear verticalmente (los baches no están al revés)
    fliplr=0.5,          # sí voltear horizontalmente
    mosaic=1.0,          # mosaic augmentation
)

print("\n🎉 Entrenamiento completado!")
print(f"📂 Resultados guardados en: {results.save_dir}")

Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.30GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Pothole-detection-using-YOLOv5-2/data.yaml, degrees=10.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_v1, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=10, pers

## **Seccion 7**
Validacion de modelo

In [ ]:
# Ver métricas
metrics = model.val()
print(f"\n📊 Métricas de validación:")
print(f"   mAP50: {metrics.box.map50:.3f}")
print(f"   mAP50-95: {metrics.box.map:.3f}")

# Ver algunas predicciones
results = model.predict(
    source='/content/Pothole-detection-using-YOLOv5-2/valid/imagess',
    conf=0.25,
    save=True,
    project='predictions',
    name='valid_set'
)

print(f"\n🖼️ Predicciones guardadas en: runs/detect/valid_set/")

Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.30GHz)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1614.9±508.4 MB/s, size: 77.1 KB)
val: Scanning /content/Pothole-detection-using-YOLOv5-2/valid/labels.cache... 133 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 133/133 273.9Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 9/9 0.4it/s 23.2s
                   all        133        341      0.489      0.623      0.538      0.314
               pothole         69        152      0.485      0.421      0.406      0.207
              pothole1         93        142       0.48      0.746      0.594      0.347
              pothole2         38         47      0.502      0.702      0.615      0.389
Speed: 2.6ms preprocess, 162.7ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /content/

FileNotFoundError: /content/Pothole-detection-using-YOLOv5-2/valid/imagess does not exist

## Seccion 8: **Optimizacion (cuantinizacion)**

Basicamente se recorta la cantidad de bits dentro de los weights

In [ ]:
from ultralytics import YOLO
from google.colab import files
import os

# Cargar tu modelo entrenado
model = YOLO('/content/pothole_detection/yolov8n_v1/weights/best.pt')

print("🔄 Exportando modelos optimizados...\n")

# ===== MODELO 1: FP16 (Float16) - Balance perfecto =====
print("📦 1/3 - Exportando FP16 (recomendado)...")
fp16_path = model.export(
    format='tflite',
    imgsz=640,
    half=True,      # FP16
    int8=False,
    nms=True
)
print(f"✅ FP16 exportado: {fp16_path}")
print(f"   Tamaño: ~3MB | Velocidad: Alta | Precisión: Excelente\n")

# ===== MODELO 2: INT8 (más pequeño) =====
print("📦 2/3 - Exportando INT8 (más rápido)...")
int8_path = model.export(
    format='tflite',
    imgsz=640,
    half=False,
    int8=True,      # Cuantización INT8
    data='/content/Pothole-detection-using-YOLOv5-2/data.yaml'  # Necesario para calibración
)
print(f"✅ INT8 exportado: {int8_path}")
print(f"   Tamaño: ~1.5MB | Velocidad: Máxima | Precisión: Buena\n")

# ===== MODELO 3: FP32 (baseline) =====
print("📦 3/3 - Exportando FP32 (baseline)...")
fp32_path = model.export(
    format='tflite',
    imgsz=640,
    half=False,
    int8=False,
    nms=True
)
print(f"✅ FP32 exportado: {fp32_path}")
print(f"   Tamaño: ~6MB | Velocidad: Normal | Precisión: Máxima\n")

# ===== DESCARGAR TODOS LOS MODELOS =====
print("⬇️ Descargando modelos...\n")

models_to_download = [
    fp16_path,
    int8_path,
    fp32_path
]

for model_path in models_to_download:
    if os.path.exists(model_path):
        print(f"📥 Descargando: {os.path.basename(model_path)}")
        files.download(model_path)
    else:
        print(f"❌ No encontrado: {model_path}")

print("\n🎉 ¡Todos los modelos descargados!")
print("\n📊 Comparación:")
print("┌──────────┬─────────┬──────────┬───────────┐")
print("│ Modelo   │ Tamaño  │ Velocidad│ Precisión │")
print("├──────────┼─────────┼──────────┼───────────┤")
print("│ FP16     │ ~3MB    │ ⚡⚡⚡    │ ⭐⭐⭐⭐  │")
print("│ INT8     │ ~1.5MB  │ ⚡⚡⚡⚡   │ ⭐⭐⭐    │")
print("│ FP32     │ ~6MB    │ ⚡⚡      │ ⭐⭐⭐⭐⭐ │")
print("└──────────┴─────────┴──────────┴───────────┘")
print("\n💡 Recomendación: Empieza con FP16, es el mejor balance")

🔄 Exportando modelos optimizados...

📦 1/3 - Exportando FP16 (recomendado)...
Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.30GHz)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/pothole_detection/yolov8n_v1/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 300, 6) (6.0 MB)

TensorFlow SavedModel: starting export with tensorflow 2.19.0...

ONNX: starting export with onnx 1.19.1 opset 22...
ONNX: slimming with onnxslim 0.1.72...
ONNX: export success ✅ 1.3s, saved as '/content/pothole_detection/yolov8n_v1/weights/best.onnx' (11.8 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.28.2...
Saved artifact at '/content/pothole_detection/yolov8n_v1/weights/best_saved_model'. The following endpoints are available:

* Endpoint 'serving_default'
  inputs_0 (POSITIONAL_ONLY): TensorSpec(shape=(1, 640, 640, 3), dtype=tf.float32, name='images')
Output Type:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

❌ No encontrado: /content/pothole_detection/yolov8n_v1/weights/best_saved_model/best_int8.tflite
📥 Descargando: best_float32.tflite


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🎉 ¡Todos los modelos descargados!

📊 Comparación:
┌──────────┬─────────┬──────────┬───────────┐
│ Modelo   │ Tamaño  │ Velocidad│ Precisión │
├──────────┼─────────┼──────────┼───────────┤
│ FP16     │ ~3MB    │ ⚡⚡⚡    │ ⭐⭐⭐⭐  │
│ INT8     │ ~1.5MB  │ ⚡⚡⚡⚡   │ ⭐⭐⭐    │
│ FP32     │ ~6MB    │ ⚡⚡      │ ⭐⭐⭐⭐⭐ │
└──────────┴─────────┴──────────┴───────────┘

💡 Recomendación: Empieza con FP16, es el mejor balance
